# Оценка баланса реактивной мощности

## Проверка по условию выполнения БРМ в сети и определение приведенных к шинам высшего напряжения нагрузок подстанции 

In [35]:
from IPython.display import Math, display
import numpy as np
%store -r Q_nb_i
%store -r S_nb_i
%store -r Lij

In [36]:
#                 K1      K2    1-3   2-3   3-4  4-5
P_1c = np.array([125.3, 119.7, 55.3, 49.7, 37.5, 17.5])
L_kolca = np.array([Lij[0, 1], Lij[0, 2], Lij[1, 3], Lij[2, 3], Lij[3, 4], Lij[4, 5]])
mask_p1c = [False, False, False, False, True, True]

In [37]:
k_odn_q = 0.98
mask_ps = [False, False, False, True, True]
Q_treb_1 = np.around(k_odn_q * np.sum(Q_nb_i), 1)
print(f'Q_treb_1 = {Q_treb_1}')

Q_treb_2 = 0.08 * np.sum(np.where(mask_ps, S_nb_i * 2, S_nb_i))
print(f'Q_treb_2 = {Q_treb_2}')

q_c0_220 = 0.14
q_c0_110 = 0.036
i = 0
Q_c_kolco = np.zeros_like(L_kolca)
for i in range(len(L_kolca)):
    if mask_p1c[i]:
        Q_c_kolco[i] = np.around(2 * q_c0_110 * L_kolca[i], 2)
    else:
        Q_c_kolco[i] = np.around(q_c0_220 * L_kolca[i], 2)
print(f'Q_c_kolco = {Q_c_kolco}')

Q_treb_1 = 93.5
Q_treb_2 = 27.48
Q_c_kolco = [5.85 6.89 6.89 5.85 1.86 1.86]


In [38]:
P_nat_220 = 130
P_nat_110 = 30
i = 0
delta_Q_l = np.zeros_like(Q_c_kolco)
for i in range(len(Q_c_kolco)):
    if mask_p1c[i] and P_1c[i] > P_nat_110:
        delta_Q_l[i] = np.around((P_1c[i] / P_nat_110)**2 * Q_c_kolco[i], 2)
    elif mask_p1c[i] and P_1c[i] <= P_nat_110:
        delta_Q_l[i] = Q_c_kolco[i]
    else:
        delta_Q_l[i] = np.around((P_1c[i] / P_nat_220)**2 * Q_c_kolco[i], 2)

Q_treb_3 = np.around(np.sum(delta_Q_l - Q_c_kolco), 1)

Q_treb_sum = np.around(Q_treb_1 + Q_treb_2 + Q_treb_3, 1)

print(f'Q_c_kolco = {Q_c_kolco}')
print(f'Q_treb_3 = {Q_treb_3} \nQ_treb_sum = {Q_treb_sum}')

Q_c_kolco = [5.85 6.89 6.89 5.85 1.86 1.86]
Q_treb_3 = -11.0 
Q_treb_sum = 110.0


In [39]:
print(f'L = {L_kolca} \nP_1c = {P_1c} \nQ_c = {Q_c_kolco} \ndelta_Q_l = {delta_Q_l}')
print(np.sum(delta_Q_l) - delta_Q_l[5])

L = [41.8 49.2 49.2 41.8 25.9 25.9] 
P_1c = [125.3 119.7  55.3  49.7  37.5  17.5] 
Q_c = [5.85 6.89 6.89 5.85 1.86 1.86] 
delta_Q_l = [5.43 5.84 1.25 0.86 2.91 1.86]
16.29


## Расстановка КУ

In [103]:
%store -r P_nb
%store -r tg_nb_i
q_rasp = 87
Q_dop_sum = Q_treb_sum - q_rasp
tg_phi_b = np.around((np.sum(Q_nb_i) - Q_dop_sum) / np.sum(P_nb), 3) # высчитываем для всех подстанций

print(tg_phi_b)
print(Q_dop_sum)
print(tg_nb_i)

0.296
23.0
[0.391 0.391 0.377 0.392 0.389]


In [98]:
Q_comp_dop = np.around(P_nb * (tg_nb_i - tg_phi_b), 2)
print("Q_comp_dop до округления вручную = ", Q_comp_dop)

Q_comp_dop = np.array([6.65, 6.65, 2.5, 3.9, 3.3]) # Вручную округляем так, чтобы сходилась сумма Qкуi (Q_comp_dop) и Qку_суммарное (Q_dop_sum) 
print("Q_comp_dop = ", Q_comp_dop)

N_bsk = np.around(Q_comp_dop / 1.2, 2)
N_bsk_treb = np.sum(Q_comp_dop) / 1.2

print("N_bsk до округления = ", N_bsk)

N_bsk_sum = 20

print("N_bsk_treb = ", N_bsk_treb)

N_bsk = np.array([6, 6, 2, 4, 2]) # уточняем количество БСК вручную

print("N_bsk_sum = ", N_bsk_sum)
print("N_bsk = ", N_bsk, "\nN_bsk = N_bsk_sum? ---", np.sum(N_bsk) == N_bsk_sum)

Q_comp_dop до округления вручную =  [6.65 6.65 2.43 3.84 3.26]
Q_comp_dop =  [6.65 6.65 2.5  3.9  3.3 ]
N_bsk до округления =  [5.54 5.54 2.08 3.25 2.75]
N_bsk_treb =  19.166666666666668
N_bsk_sum =  20
N_bsk =  [6 6 2 4 2] 
N_bsk = N_bsk_sum? --- True


In [99]:
Q_bsk_dop = N_bsk * 1.2
Q2_nb = np.around(Q_nb_i - Q_bsk_dop, 1) # здесь Q2_nb это Q''_нбi и далее двоечка будет означать апостроф
S2_nb = np.around(np.sqrt(P_nb**2 + Q2_nb**2), 1)
Q_priv = np.around(Q2_nb + 0.08 * S2_nb, 1)
S_priv = np.around(np.sqrt(P_nb**2 + Q_priv**2), 1)

Q_priv[3-1] = np.around(Q2_nb[3-1] + Q_priv[3] + Q_priv[4] + 0.08 * np.sqrt((P_nb[2] + P_nb[3] + P_nb[4])**2 + (Q2_nb[3-1] + Q_priv[3] + Q_priv[4])**2), 1)
S_priv[3-1] = np.around(np.sqrt((P_nb[2] + P_nb[3] + P_nb[4])**2 + Q_priv[2]), 1)

print("Q_bsk_dop = ", Q_bsk_dop, "\nQ''_nb = ", Q2_nb, "\nS''_nb = ", S2_nb, "\nQ_priv = ", Q_priv, "\nS_priv = ", S_priv)

Q_bsk_dop =  [7.2 7.2 2.4 4.8 2.4] 
Q''_nb =  [20.2 20.2  8.9 10.9 11.2] 
S''_nb =  [72.9 72.9 31.3 41.5 36.7] 
Q_priv =  [26.  26.  46.1 14.2 14.1] 
S_priv =  [ 74.7  74.7 105.2  42.4  37.7]


In [102]:
np.sum(Q2_nb)

71.39999999999999

In [89]:
display(Math(f'''Q_{{прив3}} = Q_{{нб3}}^{{''}} + Q_{{прив4}} + Q_{{прив5}} + 0,08 \sqrt{{(P_{{нб3}} + P_{{нб4}} + P_{{прив5}})^2 + (Q_{{нб3}}^{{''}} + Q_{{прив4}} + Q_{{прив5}})^2}} = 
             {Q2_nb[3-1]} + {Q_priv[3]} + {Q_priv[4]} + 0,08 \sqrt{{({P_nb[2]} + {P_nb[3]} + {P_nb[4]})^2 + ({Q2_nb[3-1]} + {Q_priv[3]} + {Q_priv[4]})^2}} = {Q_priv[3-1]}'''))

<IPython.core.display.Math object>

In [90]:
display(Math(f'''S_{{прив3}} = \sqrt{{(P_{{нб3}} + P_{{нб4}} + P_{{нб5}})^2 + (Q_{{прив3}})^2}} = 
                \sqrt{{({P_nb[2]} + {P_nb[3]} + {P_nb[4]})^2 + ({Q_priv[2]})^2}} = {S_priv[3-1]}'''))

<IPython.core.display.Math object>